In [1]:
import argparse
import glob
import os
import time

import numpy as np
import pandas as pd
import cv2

In [2]:
parser = {
    'val_path': '../data/val768/',
    'val_masks_path': '../data/val768_masks/',
    'test_path': '../data/test_hq/',
    'output_paths': ['20170921/output_tn/'],
    'log_every': 1000,
}
args = argparse.Namespace(**parser)

args.output_paths = ['../intermediate/'+x for x in args.output_paths]
args.img_names = sorted([x.split('/')[-1]
                         for x in glob.glob(args.test_path + '/*.jpg')])

In [3]:
def rle_encode(mask_image):
    pixels = mask_image.flatten()
    # We avoid issues with '1' at the start or end (at the corners of 
    # the original image) by setting those pixels to '0' explicitly.
    # We do not expect these to be non-zero for an accurate mask, 
    # so this should not harm the score.
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] = runs[1::2] - runs[:-1:2]
    return runs

In [ ]:
def get_mask(output_paths, index, weights=None):
    if weights is None:
        weights = [1] * len(output_paths)
        
    img = 0
    for i, path in enumerate(output_paths):
        file = os.path.join(path, str(index)+'.jpg')
        img = img + cv2.imread(file, cv2.IMREAD_GRAYSCALE) * weights[i]
    img = img / sum(weights)
    
    return img > 127.5

In [ ]:
list_rle = []
init_time = time.time()
for i in range(1, len(args.img_names)+1):
    mask = get_mask(args.output_paths, i)
    rle = rle_encode(mask)
    list_rle.append(' '.join(str(x) for x in rle))
    
    if i % args.log_every == 0:
        print("Done {:6} images in {:.0f}s".format(i, time.time()-init_time))

Done   1000 images in 39s
Done   2000 images in 78s
Done   3000 images in 116s
Done   4000 images in 155s


In [ ]:
df = pd.DataFrame({"img": args.img_names, "rle_mask": list_rle})
submiss_path = '../intermediate/output/submission.csv'
df.to_csv(submiss_path, index=False)